**Get Headlines from GDELT API, Translate, Clean, Get Sentiment / Polarity, and Store in Database**

In [1]:
reset -fs

In [2]:
import pandas as pd
import numpy  as np

import time, requests, re

from sqlalchemy  import create_engine
from textblob    import TextBlob
from googletrans import Translator, constants
from bs4         import BeautifulSoup

In [3]:
start_year  = 2021 #YYYY
end_year    = 2021 #YYYY
start_month = 9   #MM
end_month   = 9   #MM
start_day   = 1    #DD
end_day     = 1  #DD

In [4]:
engine       = create_engine("sqlite:///hl_example.db")
translator   = Translator()
alphanumeric = lambda x: re.sub(r'[^A-Za-z ]+', '', x)
lowcase      = lambda x: x.lower()

In [5]:
def get_links(sdt, edt): # YYYYMMDDHHMMSS
    url = 'https://api.gdeltproject.org/api/v2/doc/doc?query="COP26"&mode=artlist&maxrecords=250&startdatetime='+sdt+'&enddatetime='+edt
    
    response = requests.get(url)
    page = response.text
    
    time.sleep(1)
    
    return page

In [6]:
for year in range(start_year, end_year + 1, 1):
    
    for month in range(start_month, end_month + 1, 1):
    
        for day in range(start_day, end_day + 1, 1):

            for hour in range(0, 24, 1):
                
                links=pd.DataFrame(columns=['url',
                                            'arttitle',
                                            'source',
                                            'date_time', 
                                            'language', 
                                            'country']
                                  )

                start  = str.zfill(str(hour),2)
                days   = str.zfill(str(day),2)
                months = str.zfill(str(month),2)
                years  = str.zfill (str(year),4)
                
                sdt    = years + months + days + start + '0000'
                edt    = years + months + days + start + '5959'

                page = get_links(sdt, edt)

                soup = BeautifulSoup(page, "html5lib")

                for index, link in enumerate(soup.find_all('a')):
                    urls = link.get("href")
                    links.loc[index,'url'] = urls

                for index, span in enumerate(soup.find_all("span", class_="arttitle")):
                    arttitles = span.text
                    links.loc[index,'arttitle'] = arttitles

                for index, span in enumerate(soup.find_all("span", class_="sourceinfo")):
                    sourceinfos = span.text.split()
                    links.loc[index,'source'] = sourceinfos[0]
                    links.loc[index,'language'] = sourceinfos[5]
                    try:
                        try:
                            links.loc[index,'country'] = sourceinfos[6] + " " + sourceinfos[7]
                        except:
                            links.loc[index,'country'] = sourceinfos[6]
                    except:
                        pass

                script = soup.select_one('script:contains("sourceinfo_date")')
                date = re.findall(r'[0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9] [0-9][0-9]:[0-9][0-9]',script.text)
                links['date_time'] = date
                pd.to_datetime(links['date_time'])
                  
                links['arttitle_en'] = links['arttitle']
                
                for index in range(len(links)):
                    if links['language'].iloc[index] != 'English':
                        try:
                            text = links['arttitle'].iloc[index]
                            translation = translator.translate(text)
                            links['arttitle_en'].iloc[index] = translation.text
                        except:
                            pass
                    else:
                        pass
                    
                links['clean'] = links['arttitle_en']
                links['clean'] = links.clean.map(alphanumeric)
                links['clean'] = links.clean.map(lowcase)
                
                links['sentiment'] = np.nan
                links['polarity']  = np.nan
                
                for index, hl in enumerate(links['clean']):

                    links.loc[index,'sentiment'] = TextBlob(hl).sentiment.subjectivity
                    links.loc[index,'polarity'] = TextBlob(hl).sentiment.polarity

                links.to_sql('headlines', con = engine, schema=None, if_exists='append', index=True, index_label=None, chunksize=None, dtype=None, method=None)

/Applications/anaconda3/lib/python3.8/site-packages/soupsieve/css_parser.py:813: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(


**Vectorize Headline Excluding Stop Words and Determine Topics**

In [7]:
reset -fs

In [8]:
import pandas as pd
import numpy  as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition           import NMF
from sqlalchemy                      import create_engine

In [9]:
engine = create_engine("sqlite:///hl_example.db")

In [10]:
arttitle_cl = pd.read_sql('SELECT clean, country FROM headlines;', engine)

In [11]:
arttitle_cl['clean'] = arttitle_cl['clean'].str.replace('(?:biden|obama|greta|boris|cop|climate|modi|william|charles|merkel|conference|news|john|xi|terry|envoy|chief|jinping|mccrann|minister|sturgeon|nicola|pm|prince|sadyr|japarov|catriona|stewart|world|leaders|morrison|kerry|summit|change|glasgow|president|elizabeth|queen|pope|francis|johnson|thunberg|iain|macwhirter)', '')

<ipython-input-11-808e98acd981>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  arttitle_cl['clean'] = arttitle_cl['clean'].str.replace('(?:biden|obama|greta|boris|cop|climate|modi|william|charles|merkel|conference|news|john|xi|terry|envoy|chief|jinping|mccrann|minister|sturgeon|nicola|pm|prince|sadyr|japarov|catriona|stewart|world|leaders|morrison|kerry|summit|change|glasgow|president|elizabeth|queen|pope|francis|johnson|thunberg|iain|macwhirter)', '')


In [12]:
# Incorporate stop words when creating the count vectorizer

vec = TfidfVectorizer(stop_words='english', min_df = 10)
X = vec.fit_transform(arttitle_cl.clean)

In [13]:
news_tokens = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

In [14]:
news_words = news_tokens.columns.tolist()

In [15]:
nmf_model = NMF(20)
news_topics = nmf_model.fit_transform(news_tokens)

/Applications/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [16]:
def display_topics(model, feature_names, no_top_words):
    
    topics=pd.DataFrame(columns=['topic'])
    
    for index, topic in enumerate(model.components_):           
        topics.loc[index,'topic'] = '{}: '.format(index) + ' '.join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
    return topics

In [17]:
map_topics = pd.DataFrame(news_topics.round(5),
             columns = display_topics(nmf_model, vec.get_feature_names(), 10))

In [18]:
columns = map_topics.columns.values.tolist()

In [19]:
columns_str = []

for c in columns:
    columns_str.append(c[0])

In [20]:
map_topics.columns = columns_str

In [21]:
top_topic = map_topics.idxmax(axis=1)

In [22]:
news_hl_topic = pd.concat([arttitle_cl, top_topic], axis=1)
news_hl_topic.rename(columns={0:'Topic'},inplace = True)

In [23]:
news_hl_topic.to_sql('topics', con = engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None, method=None)